In [1]:
import pandas as pd
import os

In [31]:
track_df = pd.read_csv("C:/Users/aminp/OneDrive - NTNU/Dokumenter/NTNU/5år/Store, distribuerte datamengder/Store_distribuerte_datamengder/dataset/trackpoint_simen.csv")
track_df.drop("Unnamed: 0", axis = 1, inplace = True)


In [40]:
#Transformations and fixing types of columns 

track_df['user'] = track_df['user'].astype(str).str.zfill(3)
track_df["activity"] = track_df['activity'].astype(str)
track_df["time_string"] = track_df["time_string"].str.replace('\n', '')
track_df["date_string"] = track_df["date_string"].str.replace("-", "/")
track_df["date_time"] = track_df["date_string"] + " " + track_df["time_string"]
track_df['date_time'] = pd.to_datetime(track_df['date_time'], format='%Y-%m-%d %H:%M:%S')
track_df

,user,activity,lat,lon,altitude,date_days,date_string,time_string,date_time
0,000,20090412073303,40.000017,116.327479,105.000000,39915.314618,2009/04/12,07:33:03,2009-04-12 07:33:03
1,000,20090412073303,40.000168,116.327474,80.000000,39915.314688,2009/04/12,07:33:09,2009-04-12 07:33:09
2,000,20090412073303,40.000055,116.327454,99.000000,39915.314745,2009/04/12,07:33:14,2009-04-12 07:33:14
3,000,20090412073303,40.000021,116.327407,109.000000,39915.314803,2009/04/12,07:33:19,2009-04-12 07:33:19
4,000,20090412073303,40.000035,116.327281,111.000000,39915.314861,2009/04/12,07:33:24,2009-04-12 07:33:24
...,...,...,...,...,...,...,...,...,...
9681751,181,20071208010931,39.866483,116.415383,249.343832,39424.298426,2007/12/08,07:09:44,2007-12-08 07:09:44
9681752,181,20071208010931,39.866033,116.415683,173.884514,39424.299537,2007/12/08,07:11:20,2007-12-08 07:11:20
9681753,181,20071208010931,39.865550,116.415733,187.007874,39424.300486,2007/12/08,07:12:42,2007-12-08 07:12:42
9681754,181,20071208010931,39.864883,116.415750,226.377953,39424.301412,2007/12/08,07:14:02,2007-12-08 07:14:02


In [3]:
data_path = "C:/Users/aminp/OneDrive - NTNU/Dokumenter/NTNU/5år/Store, distribuerte datamengder/Store_distribuerte_datamengder/dataset/"
data_path2 = "C:/Users/aminp/OneDrive - NTNU/Dokumenter/NTNU/5år/Store, distribuerte datamengder/Store_distribuerte_datamengder/dataset/Data"


In [4]:
#Function to get all user_id and check if they have label

def get_user_id(data_path): 
    directories_data = os.listdir(f'{data_path}/Data')
    labeled_ids_path = f'{data_path}/labeled_ids.txt'
    
    with open(labeled_ids_path, "r") as file:
        lines = file.readlines()
    ids = [line.strip().split()[0] for line in lines]

    df= pd.DataFrame({"id": directories_data})
    labeled_ids_df = pd.DataFrame({"LabelID":ids})

    merged_df = pd.merge(df, labeled_ids_df, left_on="id", right_on="LabelID", how = "left")
    merged_df["has_labels"] = merged_df["LabelID"].notna()
    merged_df.drop("LabelID", axis=1, inplace=True)

    return merged_df



In [5]:
print(get_user_id(data_path))

      id  has_labels
0    000       False
1    001       False
2    002       False
3    003       False
4    004       False
..   ...         ...
177  177       False
178  178       False
179  179        True
180  180       False
181  181       False

[182 rows x 2 columns]


In [6]:
#Checking if all trajectories is unique for each user

# Initializing a dictionary to store the lists
lists_by_directory = {}

for i in range(182): 
    directory_name = f"{i:03d}"  # Format the directory name to have leading zeros if needed

   
    files_in_directory = os.listdir(os.path.join(data_path2, directory_name, "Trajectory"))

    lists_by_directory[directory_name] = files_in_directory
    

# Check if all values in each list are unique
for directory_name, file_list in lists_by_directory.items():
    is_unique = len(file_list) == len(set(file_list))
    print(f"Directory {directory_name}: All values are unique - {is_unique}")





Directory 000: All values are unique - True
Directory 001: All values are unique - True
Directory 002: All values are unique - True
Directory 003: All values are unique - True
Directory 004: All values are unique - True
Directory 005: All values are unique - True
Directory 006: All values are unique - True
Directory 007: All values are unique - True
Directory 008: All values are unique - True
Directory 009: All values are unique - True
Directory 010: All values are unique - True
Directory 011: All values are unique - True
Directory 012: All values are unique - True
Directory 013: All values are unique - True
Directory 014: All values are unique - True
Directory 015: All values are unique - True
Directory 016: All values are unique - True
Directory 017: All values are unique - True
Directory 018: All values are unique - True
Directory 019: All values are unique - True
Directory 020: All values are unique - True
Directory 021: All values are unique - True
Directory 022: All values are un

In [33]:
#Function for getting the content in labels.txt for those user which have has_label = True
def get_labels(path): 
    labels_df = labels_df = pd.DataFrame(columns=["User","Start_time", "End_time", "Transportation_mode"])
    user_df = get_user_id(data_path)
    folders_with_labels = user_df[user_df["has_labels"]]
    
    for index, row in folders_with_labels.iterrows(): 
        folder_id = row["id"]
    
        labels_file_path = os.path.join(path, folder_id, "labels.txt").replace("\\", "/")
        

        if os.path.isfile(labels_file_path): 
            labels_data = pd.read_csv(labels_file_path, sep = "\t", names=['Start_time', 'End_time', 'Transportation_mode'])
            labels_data["User"] = folder_id
            labels_df = pd.concat([labels_df, labels_data], ignore_index=True)
            return_df = labels_df.iloc[1:]

    return return_df


In [77]:
label_df = get_labels(data_path2)
label_df


,User,Start_time,End_time,Transportation_mode
1,010,2007/06/26 11:32:29,2007/06/26 11:40:29,bus
2,010,2008/03/28 14:52:54,2008/03/28 15:59:59,train
3,010,2008/03/28 16:00:00,2008/03/28 22:02:00,train
4,010,2008/03/29 01:27:50,2008/03/29 15:59:59,train
5,010,2008/03/29 16:00:00,2008/03/30 15:59:59,train
...,...,...,...,...
14782,179,2008/11/17 06:59:58,2008/11/17 07:06:16,bus
14783,179,2008/11/17 07:06:16,2008/11/17 07:14:32,walk
14784,179,2008/11/29 01:58:05,2008/11/29 02:01:39,bus
14785,179,2008/11/29 02:01:39,2008/11/29 02:07:57,walk


In [69]:
#Findig those rows which have invalid format on start/end time 

incorrect_start_times = []
incorrect_end_times = []

for index, row in label_df.iterrows():
    try:
        pd.to_datetime(row['Start_time'], format="%Y/%m/%d %H:%M:%S")
    except ValueError:
        incorrect_start_times.append(index)

    try:
            pd.to_datetime(row['End_time'], format="%Y/%m/%d %H:%M:%S")
    except ValueError:
            incorrect_end_times.append(index)

print(incorrect_start_times)
print(incorrect_end_times)

[435, 659, 678, 1104, 1125, 1159, 1193, 1195, 1198, 1981, 2050, 2257, 2374, 3344, 3366, 3421, 3931, 3943, 4038, 4053, 4129, 4249, 4672, 5971, 5980, 5987, 5989, 6024, 6195, 6239, 6359, 6388, 6417, 6436, 6443, 6501, 6590, 6623, 6627, 6633, 6644, 6665, 6702, 6923, 6931, 7071, 7075, 7091, 7097, 7099, 7253, 7722, 8660, 8677, 8685, 8723, 8762, 8866, 8876, 9084, 10208, 10263, 10302, 13485, 14430, 14446, 14454, 14467]
[435, 659, 678, 1104, 1125, 1159, 1193, 1195, 1198, 1981, 2050, 2257, 2374, 3344, 3366, 3421, 3931, 3943, 4038, 4053, 4129, 4249, 4672, 5971, 5980, 5987, 5989, 6024, 6195, 6239, 6359, 6388, 6417, 6436, 6443, 6501, 6590, 6623, 6627, 6633, 6644, 6665, 6702, 6923, 6931, 7071, 7075, 7091, 7097, 7099, 7253, 7722, 8660, 8677, 8685, 8723, 8762, 8866, 8876, 9084, 10208, 10263, 10302, 13485, 14430, 14446, 14454, 14467]


In [71]:
#This is the rows that had wrong format
mask = label_df.index.isin(incorrect_start_times)
wrong_df = label_df[mask]
wrong_df

,User,Start_time,End_time,Transportation_mode
435,020,Start Time,End Time,Transportation Mode
659,021,Start Time,End Time,Transportation Mode
678,052,Start Time,End Time,Transportation Mode
1104,053,Start Time,End Time,Transportation Mode
1125,056,Start Time,End Time,Transportation Mode
...,...,...,...,...
13485,167,Start Time,End Time,Transportation Mode
14430,170,Start Time,End Time,Transportation Mode
14446,174,Start Time,End Time,Transportation Mode
14454,175,Start Time,End Time,Transportation Mode


In [76]:
#Formating from string to datetime object so comparisons can be made. Those invalid rows have NaT (Not a Time)
label_df['Start_time'] = pd.to_datetime(label_df['Start_time'], format="%Y/%m/%d %H:%M:%S", errors='coerce')
label_df['End_time'] = pd.to_datetime(label_df['End_time'], format="%Y/%m/%d %H:%M:%S", errors='coerce')
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14786 entries, 1 to 14786
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   User                 14786 non-null  object        
 1   Start_time           14718 non-null  datetime64[ns]
 2   End_time             14718 non-null  datetime64[ns]
 3   Transportation_mode  14786 non-null  object        
dtypes: datetime64[ns](2), object(2)
memory usage: 462.2+ KB


In [65]:
activity_df = pd.DataFrame(columns=["id","User", "transportation_mode", "start_date_time", "end_date_time"])
activity_df


,id,User,transportation_mode,start_date_time,end_date_time
